# Atlantic-Eq Mode Study

In [ ]:
#Librerías Estándar 
import random
import warnings

# Análisis de Datos y Matemáticas 
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
from scipy.interpolate import RegularGridInterpolator
from netCDF4 import Dataset as ncread  # Asumiendo que usas ncread explícitamente

# Visuals
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#Utilidades y Entorno (Jupyter/IPython) ---
from tabulate import tabulate
from IPython.display import display, HTML, Image  # Todo en una sola línea
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [ ]:
path = 'E:/TFG/Datos/Importants/'
data1 = path+'HadISST_sst_v1.1_196001_202105.nc'#database_a: HadISST (Hadley Centre Sea Ice and Sea Surface Temperature dataset) de 1960-2021, 1°x1°
data2 = path+'sst.mnmean_v5_196001_202105.nc'   #database_b: ERSST (Extended Reconstructed SST), 2°x2° from NOAA
data3 = path+'noaa.pcp.mon.anom_196001_202105.nc' #datobase1: NOAA (National Oceanic and Atmospheric Administration ), 1.875°x2° 
data4 = path+'pcp.mon.ncep-ncar_196001_202105.nc' #datobase2: NCEP-NCAR, 2.5°x2.5°
#Retrieve data
nc1 = ncread(data1, 'r')  #sst
nc2 = ncread(data2, 'r')  #sst
nc3 = ncread(data3, 'r')  #prec
nc4 = ncread(data4, 'r')  #prec
#vamos a ver las variables que tenemos que llamar de cada archivo
var1 = nc1.variables["sst"]
var2 = nc2.variables["sst"]
var3 = nc3.variables["precip"]
var4 = nc4.variables["prate"]

import numpy as np

def summarize_var(var, nt=None, p=(1, 99)):
    x = var[:] if nt is None else var[:nt]
    if isinstance(x, np.ma.MaskedArray):
        masked = int(np.sum(x.mask))
        x = np.ma.filled(x, np.nan)
    else:
        masked = 0
    x = x.astype(float)
    vals = x[np.isfinite(x)]
    
    return {"units": getattr(var, "units", None),
        "p1": np.percentile(vals, p[0]) if vals.size else np.nan,
        "p99": np.percentile(vals, p[1]) if vals.size else np.nan,
        "min": np.min(vals) if vals.size else np.nan,
        "max": np.max(vals) if vals.size else np.nan,
        "n_total": x.size,
        "masked_count": masked,
        "frac_finite": vals.size / x.size if x.size else np.nan}
    
s1 = summarize_var(var1, nt=732)
s2 = summarize_var(var2, nt=732)
s3 = summarize_var(var3, nt=732)
s4 = summarize_var(var4, nt=732)
# Vamos a crear una tabla para mostrar la naturaleza de los datos:

vars_dict = {"HadISST SST": var1,"ERSST SST": var2,"NOAA PCP": var3,"NCEP-NCAR PCP": var4}
table = []
for name, var in vars_dict.items():
    s = summarize_var(var, nt=732)
    table.append([name,s["units"],(s["p1"], s["p99"]),(s["min"], s["max"]),s["frac_finite"]])
    columns = ["Base de datos","Unidades","p1 / p99","Min / Max","Fracción válida"]

df_table = pd.DataFrame(table, columns=columns)
df_table = df_table.set_index("Base de datos")

df_table["p1 / p99"] = df_table["p1 / p99"].apply(lambda x: f"({x[0]:.2f}, {x[1]:.2f})")

df_table["Min / Max"] = df_table["Min / Max"].apply(lambda x: f"({x[0]:.2f}, {x[1]:.2f})")

df_table["Fracción válida"] = df_table["Fracción válida"].apply(lambda x: f"{x:.2f}")
print(tabulate(df_table, headers="keys", tablefmt="pretty"))



In [ ]:
data_files = {"HadISST": data1,"ERSST": data2,"NOAA PCP": data3,"NCEP-NCAR": data4}

# Función para extraer info de los datos, resolución, rango, etc
def ex_info(file_path):
    
    nc = ncread(file_path, "r")  # Abrir archivo NetCDF

     # Verificamos cómo están nombradas las variables de latitud y longitud
    lat_name = next(var for var in nc.variables.keys() if "lat" in var.lower())  #next devuele el primer elemento del iterador 
    lon_name = next(var for var in nc.variables.keys() if "lon" in var.lower())  #util porque no queremos una lista de 1 elemento queremos el string
                                                                                     #lat_name[0] tambien devuelve el primero pero el next te evita
    latitudes = nc.variables[lat_name][:]                                        #crear la lista, es mas eficiente
    longitudes = nc.variables[lon_name][:]

    # Rango de latitud y longitud
    lat_range = (latitudes.min(), latitudes.max())
    lon_range = (longitudes.min(), longitudes.max())

    # Resolución (suponemos que la diferencia entre dos puntos consecutivos es la resolución)
    lat_res = round(abs(latitudes[1] - latitudes[0]),3)
    lon_res = round(abs(longitudes[1] - longitudes[0]),3)

    nc.close()  # Cerrar el archivo para liberar memoria
    return (lat_res, lon_res), lat_range, lon_range

# Crear la tabla con los datos extraídos
table = []
for name, path in data_files.items(): #Este bucle recorre cada par (nombre, ruta) del diccionario.
    info = ex_info(path)  #Pathh en este contexto seria el segundo numero de la dupla es decir data1,data2,etc
    if info:
        table.append([name, *info]) #con el * desempacamos la tupla y la tabla es [HadISST, (1,2),(3,4),(5,6)] y no [HadISST,[(1,2),(3,4),(5,6)]]
        
# Crear DataFrame
columns = ["Base de datos", "Resolución (lat, lon)", "Rango de latitud", "Rango de longitud",]
df_table = pd.DataFrame(table, columns=columns)
df_table = df_table.set_index("Base de datos")


# Hacemos bonita la tabla
df_table["Rango de latitud"] = df_table["Rango de latitud"].apply(lambda x: f"({round(float(x[0]), 2)}°, {round(float(x[1]), 2)}°)")
df_table["Rango de longitud"] = df_table["Rango de longitud"].apply( lambda x: f"({round(float(x[0]), 2)}°, {round(float(x[1]), 2)}°)")
df_table["Resolución (lat, lon)"] = df_table["Resolución (lat, lon)"].apply(lambda x: f"({round(float(x[0]), 3)}°, {round(float(x[1]), 3)}°)")

print(tabulate(df_table, headers="keys", tablefmt="pretty"))




In [ ]:
#Entendemos nuestros datos:
sst_a = nc1.variables['sst'][:732] #HadISST: R=1°x1°,
sst_b = nc2.variables['sst'][:732] #ERSSTD:  R=2°x2°   
pcp = nc3.variables['precip'][:732] #NOAA PCP:  R=2.5°x2.5°
pcp2 = nc4.variables['prate'][:732] #NCEP-NCAR:  R=1.889°x1.875°


def lat_lon(file_path): #funcion para retornar long y lat en funcion de la data
    nc = ncread(file_path, "r")
    lat_name = next(var for var in nc.variables.keys() if "lat" in var.lower()) #busca el nombre asociado las coordenadas
    lon_name = next(var for var in nc.variables.keys() if "lon" in var.lower()) #suponemos que contiene lat/lon
    longitudes = nc.variables[lon_name][:]
    latitudes = nc.variables[lat_name][:]                                        
    nc.close()
    return longitudes, latitudes
    
lon_a, lat_a  = lat_lon(data1)
lon_b, lat_b  = lat_lon(data2)
lon_pcp, lat_pcp  = lat_lon(data3)
lon_pcp2, lat_pcp2  = lat_lon(data4)

nt = len(sst_a[:,0,0])
nyr = int(nt/12) 
ny_a = len(lat_a)
ni_a = len(lon_a)   # al haber diferentes resoluciones el numeros de lat y longitud varia asi que mejor mirar todas
ny_b = len(lat_b) 
ni_b = len(lon_b)
ny_pcp = len(lat_pcp)
ni_pcp = len(lon_pcp)
ny_pcp2 = len(lat_pcp2)
ni_pcp2 = len(lon_pcp2)

sst_a = np.where(sst_a <= -1.79, np.nan, sst_a)  # SST oceánica no baja de -1.8°C
sst_a = np.where(sst_a > 40, np.nan, sst_a)    # SST oceánica no sube de ~35°C
sst_b = np.where(sst_b <= -1.79, np.nan, sst_b)  # SST oceánica no baja de -1.8°C
sst_b = np.where(sst_b > 40, np.nan, sst_b)    # SST oceánica no sube de ~35°C

# Calcular medias anuales (732 meses = 61 años)
sst_a_annual = sst_a.reshape(61, 12, *sst_a.shape[1:]).mean(axis=1)
sst_b_annual = sst_b.reshape(61, 12, *sst_b.shape[1:]).mean(axis=1)

Temp_ATL3_a = []
Temp_ATL3_b = []
Temp_Niño_a = []
Temp_Niño_b = []

for i in range(61):
    matriuATL3_a = sst_a_annual[i, 86:94, 160:180]      # HadISST
    Temp_ATL3_a.append(np.mean(matriuATL3_a))

    matriuATL3_b = sst_b_annual[i, 42:47, 170:180]      # ERSST
    Temp_ATL3_b.append(np.mean(matriuATL3_b))

    matriuNiño_a = sst_a_annual[i, 84:96, 10:60]      # HadISST
    Temp_Niño_a.append(np.mean(matriuNiño_a))

    matriuNiño_b = sst_b_annual[i, 41:47, 95:120]      # ERSST
    Temp_Niño_b.append(np.mean(matriuNiño_b))



    
Temp_ATL3_a = np.array(Temp_ATL3_a)
Temp_ATL3_b = np.array(Temp_ATL3_b)
Temp_Niño_a = np.array(Temp_Niño_a)
Temp_Niño_b = np.array(Temp_Niño_b)

# Tendencias lineales
years=(np.arange(1960, 2021, 1))
# ATL3
coef_ATL3_a = np.polyfit(years, Temp_ATL3_a, 1)
coef_ATL3_b = np.polyfit(years, Temp_ATL3_b, 1)

trend_ATL3_a = np.polyval(coef_ATL3_a, years)
trend_ATL3_b = np.polyval(coef_ATL3_b, years)

slope_ATL3_a_dec = coef_ATL3_a[0] * 10
slope_ATL3_b_dec = coef_ATL3_b[0] * 10

# Niño 3.4
coef_Niño_a = np.polyfit(years, Temp_Niño_a, 1)
coef_Niño_b = np.polyfit(years, Temp_Niño_b, 1)

trend_Niño_a = np.polyval(coef_Niño_a, years)
trend_Niño_b = np.polyval(coef_Niño_b, years)

slope_Niño_a_dec = coef_Niño_a[0] * 10
slope_Niño_b_dec = coef_Niño_b[0] * 10

# Figura con dos paneles


fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharex=True)

# ATL3
axs[0].plot(years, Temp_ATL3_a, 'o-', color='C0', alpha=0.4, label='HadISST (datos)')
axs[0].plot(years, Temp_ATL3_b, 'o-', color='C1', alpha=0.4, label='ERSST (datos)')
axs[0].plot(years, trend_ATL3_a, color='C0', lw=2.5, label='HadISST – Tendencia')
axs[0].plot(years, trend_ATL3_b, color='C1', lw=2.5, label='ERSST – Tendencia')

axs[0].set_title('Atlantic Niño (ATL3)')
axs[0].set_ylabel('SST Media Anual (°C)')
axs[0].set_xlim(1960, 2020)
axs[0].grid(True, alpha=0.3)

axs[0].text(
    0.03, 0.95,
    f"HadISST: {slope_ATL3_a_dec:.2f} °C/década\n"
    f"ERSST:  {slope_ATL3_b_dec:.2f} °C/década",
    transform=axs[0].transAxes,
    fontsize=10,
    verticalalignment='top'
)

# Niño 3.4
axs[1].plot(years, Temp_Niño_a, 'o-', color='C0', alpha=0.4, label='HadISST (datos)')
axs[1].plot(years, Temp_Niño_b, 'o-', color='C1', alpha=0.4, label='ERSST (datos)')
axs[1].plot(years, trend_Niño_a, color='C0', lw=2.5, label='HadISST – Tendencia')
axs[1].plot(years, trend_Niño_b, color='C1', lw=2.5, label='ERSST – Tendencia')

axs[1].set_title('Pacific Niño (Niño 3.4)')
axs[1].set_xlim(1960, 2020)
axs[1].grid(True, alpha=0.3)

axs[1].text(
    0.03, 0.95,
    f"HadISST: {slope_Niño_a_dec:.2f} °C/década\n"
    f"ERSST:  {slope_Niño_b_dec:.2f} °C/década",
    transform=axs[1].transAxes,
    fontsize=10,
    verticalalignment='top'
)
# Ejes comunes
for ax in axs:
    ax.set_xlabel('Años (1960–2020)')
    ax.set_xticks(np.arange(1960, 2021, 5))

plt.tight_layout()
plt.show()




![SST_mean](img/1.png)

$$
\overline{T}
=
\frac{\sum_{\varphi,\lambda} T(\varphi,\lambda)\,\cos(\varphi)}
     {\sum_{\varphi,\lambda} \cos(\varphi)}
$$

In [ ]:

def lon_to_180(lon):
    lon = np.array(lon, dtype=float)
    return ((lon + 180) % 360) - 180

def _ensure_increasing(lat, lon, field2d):
    lat = np.array(lat); lon = np.array(lon)
    if lat[0] > lat[-1]:
        lat = lat[::-1]
        field2d = field2d[::-1, :]
    lon_sort_idx = np.argsort(lon)
    lon = lon[lon_sort_idx]
    field2d = field2d[:, lon_sort_idx]
    return lat, lon, field2d

def interp2d_to_target(lat_src, lon_src, field2d, lat_t, lon_t):
    lat_src, lon_src, field2d = _ensure_increasing(lat_src, lon_src, field2d)
    interp = RegularGridInterpolator((lat_src, lon_src), field2d, method="linear",
                                    bounds_error=False, fill_value=np.nan)
    LonT, LatT = np.meshgrid(lon_t, lat_t)
    pts = np.column_stack([LatT.ravel(), LonT.ravel()])
    return interp(pts).reshape(len(lat_t), len(lon_t))

def area_weighted_mean(field2d, lat_t):
    w = np.cos(np.deg2rad(lat_t))[:, None]
    m = np.isfinite(field2d)
    return np.nan if not np.any(m) else np.nansum(field2d * w) / np.nansum(w * m)

# --- Longitudes coherentes ---
lon_a_180 = lon_to_180(lon_a)
lon_b_180 = lon_to_180(lon_b)

years = np.arange(1960, 2021, 1)
ddeg = 0.25

def series_box(lat_min, lat_max, lon_min, lon_max):
    lat_t = np.arange(lat_min, lat_max + 1e-9, ddeg)
    lon_t = np.arange(lon_min, lon_max + 1e-9, ddeg)
    A = np.empty(61); B = np.empty(61)
    for i in range(61):
        fld_a = interp2d_to_target(lat_a, lon_a_180, sst_a_annual[i, :, :], lat_t, lon_t)
        fld_b = interp2d_to_target(lat_b, lon_b_180, sst_b_annual[i, :, :], lat_t, lon_t)
        A[i] = area_weighted_mean(fld_a, lat_t)
        B[i] = area_weighted_mean(fld_b, lat_t)
    return A, B

def trend(y):
    c = np.polyfit(years, y, 1)
    return np.polyval(c, years), c[0] * 10

# ATL3: 4S–4N, 20W–0  (en -180..180)
Temp_ATL3_a_intr, Temp_ATL3_b_intr = series_box(-4, 4, -20, 0)

# Niño 3.4 (tus coordenadas): 6S–6N, 170W–120W  -> -170 .. -120
Temp_Niño_a_intr, Temp_Niño_b_intr = series_box(-6, 6, -170, -120)

trend_ATL3_a, slope_ATL3_a_dec = trend(Temp_ATL3_a_intr)
trend_ATL3_b, slope_ATL3_b_dec = trend(Temp_ATL3_b_intr)
trend_Niño_a, slope_Niño_a_dec = trend(Temp_Niño_a_intr)
trend_Niño_b, slope_Niño_b_dec = trend(Temp_Niño_b_intr)

fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharex=True)

# --- ATL3 ---
axs[0].plot(years, Temp_ATL3_a_intr, 'o-', label='HadISST interp (datos)', color='C0', alpha=0.4)
axs[0].plot(years, Temp_ATL3_b_intr, 'o-', label='ERSST interp (datos)', color='C1', alpha=0.4)
axs[0].plot(years, trend_ATL3_a, label='HadISST interp – Tendencia', color='C0', lw=2.5)
axs[0].plot(years, trend_ATL3_b, label='ERSST interp – Tendencia', color='C1', lw=2.5)
axs[0].set_title('Atlantic Niño (ATL3) – Interpolado')
axs[0].set_ylabel('SST Media Anual (°C)')
axs[0].grid(True, alpha=0.3)
axs[0].text(0.02, 0.95,
            f"HadISST: {slope_ATL3_a_dec:.2f} °C/década\nERSST:  {slope_ATL3_b_dec:.2f} °C/década",
            transform=axs[0].transAxes, fontsize=10, va="top")

# --- Niño 3.4 ---
axs[1].plot(years, Temp_Niño_a_intr, 'o-', label='HadISST interp (datos)', color='C0', alpha=0.4)
axs[1].plot(years, Temp_Niño_b_intr, 'o-', label='ERSST interp (datos)', color='C1', alpha=0.4)
axs[1].plot(years, trend_Niño_a, label='HadISST interp – Tendencia', color='C0', lw=2.5)
axs[1].plot(years, trend_Niño_b, label='ERSST interp – Tendencia', color='C1', lw=2.5)
axs[1].set_title('Pacific Niño (Niño 3.4) – Interpolado')
axs[1].grid(True, alpha=0.3)
axs[1].text(0.02, 0.95,
            f"HadISST: {slope_Niño_a_dec:.2f} °C/década\nERSST:  {slope_Niño_b_dec:.2f} °C/década",
            transform=axs[1].transAxes, fontsize=10, va="top")

for ax in axs:
    ax.set_xlabel('Años (1960–2020)')
    ax.set_xlim(1960, 2020)
    ax.set_xticks(np.arange(1960, 2021, 5))

plt.tight_layout()
plt.show()



![SST_mean_interpoled](img/2.png)

In [ ]:
# redefinimos 
sst_a = nc1.variables['sst'][:732] #HadISST: R=1°x1°
sst_b = nc2.variables['sst'][:732] #ERSST:  R=2°x2°   

def to_float_nan(a):
    """Convierte Variable/MaskedArray a ndarray float con NaNs donde haya máscara."""
    if np.ma.isMaskedArray(a):
        return np.ma.filled(a, np.nan).astype(float)
    return np.array(a, dtype=float)

def calc_monthly_anoms(fen, detrend=True):
    """
    fen: (time, lat, lon) mensual
    return: anoms (year, month, lat, lon)
    """
    arr = to_float_nan(fen)

    # Si viniera 4D (time, level, lat, lon), toma level 0
    if arr.ndim == 4:
        arr = arr[:, 0, :, :]
    if arr.ndim != 3:
        raise ValueError(f"Se esperaba (time, lat, lon), pero llegó {arr.shape}")

    nt, ny, nx = arr.shape
    nyr = nt // 12
    nt_use = nyr * 12
    dat = arr[:nt_use].copy()

    # Detrend (estable): ajusta con NaNs rellenados, resta solo donde hay dato
    if detrend:
        x = np.arange(nt_use)
        X = np.column_stack([x, np.ones(nt_use)])          # (nt, 2)
        Y = dat.reshape(nt_use, -1)                        # (nt, ngrid)
        Yf = np.nan_to_num(Y, nan=0.0)                     # para que lstsq no reviente

        coeffs = np.linalg.lstsq(X, Yf, rcond=None)[0]     # (2, ngrid)
        trend = (X @ coeffs).reshape(dat.shape)            # (nt, ny, nx)

        mask = np.isfinite(dat)
        dat[mask] = dat[mask] - trend[mask]

    # Climatología mensual y anomalías
    dat4 = dat.reshape(nyr, 12, ny, nx)                    # (year, month, lat, lon)
    clim = np.nanmean(dat4, axis=0)                        # (month, lat, lon)
    anoms = dat4 - clim[None, :, :, :]                     # (year, month, lat, lon)
    return anoms
            
#calculem les anomalies dels 4 casos, fem reshape

anoms_a0 = calc_monthly_anoms(sst_a, detrend=True)
anoms_b = calc_monthly_anoms(sst_b, detrend=True)
#en la primera tabla y leyendo la data nos damos cuenta que pcp ya viene en forma de anomalias pero hay que modificar us forma (messes,lat,lon)->(year, month, lat, lon)
mes, lat, lon = pcp.shape
yr = mes // 12
anoms_pcp = pcp.reshape(yr,12,lat,lon)
anoms_pcp2 = calc_monthly_anoms(pcp2, detrend=True) #recordar que pcp eran anomalias solo necesitamos calcular las de pcp2

def plot_anom_summary(ax, lon, lat, anoms, title, cb_label):
    # Campo diagnóstico: máximo absoluto
    field = np.nanmax(np.abs(anoms), axis=(0, 1))  # (lat, lon)

    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.gridlines(draw_labels=True)

    levels = np.linspace(0, 10, 11)

    fill = ax.contourf(
        lon, lat, field,
        levels=levels,
        transform=ccrs.PlateCarree(),
        cmap='YlOrRd',
        extend='max'
    )

    ax.set_title(title, fontsize=12)

    cb = plt.colorbar(
        fill,
        ax=ax,
        orientation='horizontal',
        pad=0.05,
        aspect=40
    )
    cb.set_label(cb_label, fontsize=12)

    # --- mini leyenda min / max ---
    vmin = np.nanmin(field)
    vmax = np.nanmax(field)

    txt = f"min = {vmin:.2f}\nmax = {vmax:.2f}"

    ax.text(
        0.02, 0.02, txt,
        transform=ax.transAxes,
        fontsize=9,
        verticalalignment='bottom',
        horizontalalignment='left',
        bbox=dict(
            facecolor='white',
            edgecolor='none',
            alpha=0.75
        )
    )

    return fill



In [ ]:
# Definimos la figura 2×2 para los 4 datasets
fig, axes = plt.subplots(
    2, 2,
    figsize=(16, 12),
    subplot_kw={'projection': ccrs.PlateCarree(central_longitude=330)}
)

plot_anom_summary(
    axes[0,0], lon_a, lat_a,
    anoms_a0,
    'HadISST anomalies: max(|anom|) over 61y×12m',
    cb_label='SST anomaly (°C)')

plot_anom_summary(
    axes[0,1], lon_b, lat_b,
    anoms_b,
    'ERSST anomalies: max(|anom|) over 61y×12m',
    cb_label='SST anomaly (°C)')

plot_anom_summary(
    axes[1,0], lon_pcp, lat_pcp,
    anoms_pcp,
    'PCP anomalies: max(|anom|) over 61y×12m',
    cb_label='PCP anomaly')

plot_anom_summary(
    axes[1,1], lon_pcp2, lat_pcp2,
    anoms_pcp2,
    'PCP2 anomalies: max(|anom|) over 61y×12m',
    cb_label='PCP2 anomaly')

plt.tight_layout()
plt.show()


![Data anomalies max](img/3.png)

In [ ]:

anoms_a = anoms_a0.copy()
bad_phys = np.abs(anoms_a0) > 10
anoms_a[bad_phys] = np.nan #eliminamos los valores que de desvian muchissimo de lo esperado para evitar outliers en anomalias, y con esto tenemos las anomalias limpias

# frecuencia absoluta: número de meses problemáticos
thr = 10
freq = np.sum(np.abs(anoms_a0) > thr, axis=(0, 1))  # (lat, lon)
n_total = anoms_a0.shape[0] * anoms_a0.shape[1]  # 61*12 = 732
freq_rel = freq / n_total

freq_c = np.sum(np.abs(anoms_a) > thr, axis=(0, 1))  # (lat, lon)
n_total = anoms_a.shape[0] * anoms_a.shape[1]  # 61*12 = 732
freq_rel_c = freq_c / n_total

def plot_freq(ax, lon, lat, freq_field, title, vmax=None):
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.gridlines(draw_labels=True)

    if vmax is None:
        vmax = np.nanmax(freq_field)
    levels = np.linspace(0, vmax, 11)

    fill = ax.contourf(
        lon, lat, freq_field,
        levels=levels,
        transform=ccrs.PlateCarree(),
        cmap='Reds',
        extend='max'
    )

    ax.set_title(title, fontsize=12)

    cb = plt.colorbar(
        fill,
        ax=ax,
        orientation='horizontal',
        pad=0.05,
        aspect=40
    )
    cb.set_label('Number of months', fontsize=12)

    # mini-leyenda min/max
    vmin = np.nanmin(freq_field)
    vmax_f = np.nanmax(freq_field)
    n_total = 61 * 12  # meses
    freq_rel_global = np.nanmean(freq_field) / n_total

    txt = (
        f"min = {vmin:.0f}\n"
        f"max = {vmax_f:.0f}\n"
        f"mean freq = {100*freq_rel_global:.2f}%"
    )
    ax.text(
        0.02, 0.02, txt,
        transform=ax.transAxes,
        fontsize=9,
        va='bottom', ha='left',
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.75)
    )

    return fill

    # Escala común para comparar
vmax_common = np.nanpercentile(np.maximum(freq, freq_c), 99)
if vmax_common < 1:
    vmax_common = np.nanmax(np.maximum(freq, freq_c))

fig, axes = plt.subplots(
    1, 2,
    figsize=(16, 5),
    subplot_kw={'projection': ccrs.PlateCarree(central_longitude=330)}
)

plot_freq(
    axes[0], lon_a, lat_a,
    freq,
    f'HadISST frequency: |anom| > {thr}°C (raw)',
    vmax=vmax_common
)

plot_freq(
    axes[1], lon_a, lat_a,
    freq_c,
    f'HadISST frequency: |anom| > {thr}°C (cleaned)',
    vmax=vmax_common
)

plt.tight_layout()
plt.show()




![Clean Outliers Haddist](img/4.png)

In [ ]:
def var_mensual(var_mon, lat_src, lon_src, lat_min, lat_max, lon_min, lon_max,
                      ddeg=0.25, reducer="area_mean"):
    lon_src_180 = lon_to_180(lon_src)

    lat_t = np.arange(lat_min, lat_max + 1e-9, ddeg)
    lon_t = np.arange(lon_min, lon_max + 1e-9, ddeg)

    out = np.empty(12, dtype=float)

    for m in range(12):
        fld = interp2d_to_target(lat_src, lon_src_180, var_mon[m, :, :], lat_t, lon_t)

        if reducer == "area_mean":
            out[m] = area_weighted_mean(fld, lat_t)
        elif reducer == "mean":
            out[m] = np.nanmean(fld)
        else:
            raise ValueError("reducer must be 'area_mean' or 'mean'")

    return out

# std mensual ya calculada
var_a = np.nanstd(anoms_a, axis=0)      # (12, lat, lon)
var_b = np.nanstd(anoms_b, axis=0)
var_pcp = np.nanstd(anoms_pcp, axis=0)
var_pcp2 = np.nanstd(anoms_pcp2, axis=0)

variacion_mensual_ATL3_a  = var_mensual(var_a, lat_a, lon_a, -4,  4,  -20,    0, ddeg=0.25)
variacion_mensual_Niño_a  = var_mensual(var_a, lat_a, lon_a, -6,  6, -170, -120, ddeg=0.25)

variacion_mensual_ATL3_b  = var_mensual(var_b, lat_b, lon_b, -4,  4,  -20,    0, ddeg=0.25)
variacion_mensual_Niño_b  = var_mensual(var_b, lat_b, lon_b, -6,  6, -170, -120, ddeg=0.25)


In [ ]:
# Mostrar el gráfico

meses = ['Jan','Feb','Mar','Abr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


# SSTxATL3 y STTxNiño3.4
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

axs[0].plot(meses, variacion_mensual_ATL3_a,'-ro', markersize=2)
axs[0].plot(meses, variacion_mensual_ATL3_b,'-bo', markersize=2)
axs[0].grid() # Agrega un fondo cuadriculado al gráfico
axs[0].set_ylabel('Variability (°C)') # Renombra el eje y como "Variability (°C)"
axs[0].set_title('ATL3') 
axs[0].set_yticks(np.linspace(0.3, 0.7, 12)) # Establece 9 divisiones en el eje y
axs[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f')) # Formatea el eje y para mostrar solo 3 decimales
axs[0].margins(x=0.0909)
axs[0].legend(['HadISST','ERSST'])


axs[1].plot(meses, variacion_mensual_Niño_a,'-ro', markersize=2)
axs[1].plot(meses, variacion_mensual_Niño_b,'-bo', markersize=2)
axs[1].grid() # Agrega un fondo cuadriculado al gráfico
axs[1].set_ylabel('Variability (°C)') # Renombra el eje y como "Variability (°C)"
axs[1].set_title('Niño3.4') 
axs[1].set_yticks(np.linspace(0.5, 1.2, 12)) # Establece 9 divisiones en el eje y
axs[1].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f')) # Formatea el eje y para mostrar solo 3 decimales
axs[1].margins(x=0.0909)
axs[1].legend(['HadISST','ERSST'])

plt.tight_layout()
plt.show()


![Variabilidad Anom_SST](img/5.png)

In [ ]:
variacion_mensual_ATL3_pcp  = var_mensual(var_pcp, lat_pcp, lon_pcp, -4,  4,  -20,    0, ddeg=0.25)
variacion_mensual_Niño_pcp  = var_mensual(var_pcp, lat_pcp, lon_pcp, -6,  6, -170, -120, ddeg=0.25)

variacion_mensual_ATL3_pcp2  = var_mensual(var_pcp2, lat_pcp2, lon_pcp2, -4,  4,  -20,    0, ddeg=0.25)
variacion_mensual_Niño_pcp2  = var_mensual(var_pcp2, lat_pcp2, lon_pcp2, -6,  6, -170, -120, ddeg=0.25)

# SSTxATL3 y STTxNiño3.4
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

axs[0].plot(meses, variacion_mensual_ATL3_pcp,'-ro', markersize=2)
axs[0].plot(meses, variacion_mensual_ATL3_pcp2,'-bo', markersize=2)
axs[0].grid() # Agrega un fondo cuadriculado al gráfico
axs[0].set_ylabel('Variability (mm/day)') # Renombra el eje y como "Variability (°C)"
axs[0].set_title('ATL3') 
axs[0].set_yticks(np.linspace(0, 2, 12)) # Establece 9 divisiones en el eje y
axs[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f')) # Formatea el eje y para mostrar solo 3 decimales
axs[0].margins(x=0.0909)
axs[0].legend(['NOAA PCP','NCEP-NCAR'])


axs[1].plot(meses, variacion_mensual_Niño_pcp,'-ro', markersize=2)
axs[1].plot(meses, variacion_mensual_Niño_pcp2,'-bo', markersize=2)
axs[1].grid() # Agrega un fondo cuadriculado al gráfico
axs[1].set_ylabel('Variability (mm/day)') # Renombra el eje y como "Variability (°C)"
axs[1].set_title('Niño3.4') 
axs[1].set_yticks(np.linspace(0.5, 3, 12)) # Establece 9 divisiones en el eje y
axs[1].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f')) # Formatea el eje y para mostrar solo 3 decimales
axs[1].margins(x=0.0909)
axs[1].legend(['NOAA PCP','NCEP-NCAR'])

plt.tight_layout()
plt.show()



![Variabilidad Anom_PCP](img/6.png)

In [ ]:
#Vamos a definir los indices globales como las anomalias entre la variabilidad
index_sst_a = anoms_a / var_a        #HadISST
index_sst_b = anoms_b / var_b        #ERSST
index_pcp = anoms_pcp / var_pcp      #NOAA PCP
index_pcp2 = anoms_pcp2 / var_pcp2   #NCEP-NCAR 

#A partir de estos indices vamos a calcular su media temporal en los meses de mayor intensidad de cada evento, tendremos como resultado matrices 2D delimitadas por las zonas
JJ_months = [5,6]
ND_months = [10, 11]

index_sst_a_JJ = np.nanmean(index_sst_a[:, JJ_months, :, :], axis=1)
index_sst_a_ND = np.nanmean(index_sst_a[:, ND_months, :, :], axis=1)

index_sst_b_JJ = np.nanmean(index_sst_b[:, JJ_months, :, :], axis=1)
index_sst_b_ND = np.nanmean(index_sst_b[:, ND_months, :, :], axis=1)

index_pcp_JJ = np.nanmean(index_pcp[:, JJ_months, :, :], axis=1)
index_pcp_ND = np.nanmean(index_pcp[:, ND_months, :, :], axis=1)

index_pcp2_JJ = np.nanmean(index_pcp2[:, JJ_months, :, :], axis=1)
index_pcp2_ND = np.nanmean(index_pcp2[:, ND_months, :, :], axis=1)

#Tambien vamos a calcular la media de estos indices en la zona de cada evento.

# Regiones (en coordenadas físicas)
ATL3_lats = (-4.0, 4.0)
ATL3_lons = (-20, 0)      # 20W–0E 

NIÑO_lats = (-6.0, 6.0)
NIÑO_lons = (-170, -120)      # 170W–120W 


def area_mean_index(season_field, lat, lon, lat_bounds, lon_bounds):
    lat = np.asarray(lat)
    lon = lon_to_180(lon)

    lat_min, lat_max = lat_bounds
    lon_min, lon_max = lon_bounds

    lat_mask = (lat >= lat_min) & (lat <= lat_max)
    lon_mask = (lon >= lon_min) & (lon <= lon_max)

    sub = season_field[:, lat_mask, :][:, :, lon_mask]
    return np.nanmean(sub, axis=(1, 2))
    
#Vamos a coger los indices en forma de matriz anteriores y les vamos a calcular la media de temperatura para cada zona especifica, este indice resultantante solo tendra dimension temporal
# Hadisst
ATL3_sst_a_index = area_mean_index(index_sst_a_JJ, lat_a, lon_a, ATL3_lats, ATL3_lons)
Niño_sst_a_index = area_mean_index(index_sst_a_ND, lat_a, lon_a, NIÑO_lats, NIÑO_lons) 

# ERSST
ATL3_sst_b_index = area_mean_index(index_sst_b_JJ, lat_b, lon_b, ATL3_lats, ATL3_lons)
Niño_sst_b_index = area_mean_index(index_sst_b_ND, lat_b, lon_b, NIÑO_lats, NIÑO_lons)  
    
# NOAA PCP
ATL3_pcp_index = area_mean_index(index_pcp_JJ, lat_pcp, lon_pcp, ATL3_lats, ATL3_lons)
Niño_pcp_index = area_mean_index(index_pcp_ND, lat_pcp, lon_pcp, NIÑO_lats, NIÑO_lons) 

# #NCEP-NCAR
ATL3_pcp2_index = area_mean_index(index_pcp2_JJ, lat_pcp2, lon_pcp2, ATL3_lats, ATL3_lons)
Niño_pcp2_index = area_mean_index(index_pcp2_ND, lat_pcp2, lon_pcp2, NIÑO_lats, NIÑO_lons)

#Una vez tenemos los indices vamos a calcular la media de anomalies en en los messes de Julio Junio y de Noviembre Diciembre, estas son globales
anoms_a_JJ = np.mean(anoms_a[:,JJ_months,:,:], axis=1)
anoms_b_JJ = np.mean(anoms_b[:,JJ_months,:,:], axis=1)
anoms_pcp_JJ = np.mean(anoms_pcp[:,JJ_months,:,:], axis=1)
anoms_pcp2_JJ = np.mean(anoms_pcp2[:,JJ_months,:,:], axis=1)

anoms_a_ND = np.mean(anoms_a[:,ND_months,:,:], axis=1)
anoms_b_ND = np.mean(anoms_b[:,ND_months,:,:], axis=1)
anoms_pcp_ND = np.mean(anoms_pcp[:,ND_months,:,:], axis=1)
anoms_pcp2_ND = np.mean(anoms_pcp2[:,ND_months,:,:], axis=1)

#Vamos a definir una funcion donde poder ver la tendencia y las correlacion entre estas anomalias en los periodos JJ y ND y su indice normalizado y medio de cada zona  
# Analizamos en qué regiones del mundo las anomalías de SST o precipitación
# covarían con el índice regional (ATL3 o Niño3.4).
# Las zonas con alta correlación indican regiones donde la variabilidad
# interanual está fuertemente acoplada al índice, por lo tanto al evento climatico

def find_a_r(anom_det, index):
    index = np.asarray(index)
    n_time = len(index)

    x = index
    x_mean = np.mean(x)
    x_centered = x - x_mean

    y = anom_det  # (time, lat, lon)
    y_mean = np.mean(y, axis=0, keepdims=True)
    y_centered = y - y_mean

    cov = np.sum(x_centered[:, np.newaxis, np.newaxis] * y_centered, axis=0) / (n_time - 1)
    var_x = np.var(x, ddof=1)
    slopes = cov / var_x

    std_y = np.std(y, axis=0, ddof=1)
    correlations = cov / (np.std(x, ddof=1) * std_y)

    return slopes, correlations
    

In [ ]:
a_HADISST_ATL3, corr_HADISST_ATL3 = find_a_r(anoms_a_JJ, ATL3_sst_a_index)

a_HADISST_NIÑO, corr_HADISST_NIÑO = find_a_r(anoms_a_ND, Niño_sst_a_index)

a_ERSST_ATL3, corr_ERSST_ATL3 = find_a_r(anoms_b_JJ, ATL3_sst_b_index)

a_ERSST_NIÑO, corr_ERSST_NIÑO = find_a_r(anoms_b_ND, Niño_sst_b_index)

a_pcp_ATL3, corr_pcp_ATL3 = find_a_r(anoms_pcp_JJ, ATL3_pcp_index)

a_pcp_NIÑO, corr_pcp_NIÑO = find_a_r(anoms_pcp_ND, Niño_pcp_index)

a_pcp2_ATL3, corr_pcp2_ATL3 = find_a_r(anoms_pcp2_JJ, ATL3_pcp2_index)

a_pcp2_NIÑO, corr_pcp2_NIÑO = find_a_r(anoms_pcp2_ND, Niño_pcp2_index)

def configure_plot(ax, title, cb_label, data_type='temp'):
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.gridlines(draw_labels=True)

    cmap = plt.cm.RdBu_r
    clev = np.linspace(-1, 1, 11)

    return cmap, clev
    
def create_plot(ax, lon, lat, slope_data, corr_data, title, cb_label, data_type='temp'):
    # --- Constantes del estudio ---
    N = 61
    alpha = 0.05
    df = N - 2

    # --- Base map ---
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.gridlines(draw_labels=True)

    cmap = plt.cm.RdBu_r

    slope_plot = np.array(slope_data, copy=True)
    corr_plot  = np.array(corr_data,  copy=True)

    # COLORES-SLOPE
    if data_type == 'temp':
        clev_slope = np.linspace(-1, 1, 11)
    else:
        vmax = np.nanpercentile(np.abs(slope_plot), 95)
        vmax = max(min(vmax, 2.0), 1e-6)
        clev_slope = np.linspace(-vmax, vmax, 11)

    fill = ax.contourf(
        lon, lat, slope_plot,
        levels=clev_slope,
        transform=ccrs.PlateCarree(),
        cmap=cmap,
        extend='both'
    )

    
    #CONTORNOS = r (tamaño de correlación)
    corr_levels = [-0.6, -0.4, -0.2, 0.2, 0.4, 0.6]
    cont = ax.contour(lon, lat, corr_plot,levels=corr_levels,transform=ccrs.PlateCarree(),colors='k',linewidths=0.7,alpha=0.7)
    ax.clabel(cont, inline=True, fontsize=8, fmt="%.1f")

    
    #HACHURADO = p < 0.05 (significancia de r)
    r = np.clip(corr_plot, -0.999999, 0.999999)
    t_stat = r * np.sqrt(df / (1.0 - r**2))
    pval = 2.0 * stats.t.sf(np.abs(t_stat), df)
    sig = (pval < alpha).astype(int)  # 1 significativo
    ax.contourf(lon, lat, sig,levels=[0.5, 1.5],transform=ccrs.PlateCarree(),colors='none',hatches=['....'])  # cambia a '///' si lo prefieres)

    # --- Título y colorbar ---
    ax.set_title(title, fontsize=12)

    cb = plt.colorbar(
        fill,
        ax=ax,
        orientation='horizontal',
        pad=0.05,
        aspect=40
    )
    cb.set_label(cb_label, fontsize=12)

    return fill

In [ ]:
# SSTxATL3 y STTxNiño3.4
fig, axes = plt.subplots(
    2, 2,
    figsize=(16, 12),
    subplot_kw={'projection': ccrs.PlateCarree(central_longitude=330)}
)

create_plot(
    axes[0,0], lon_a, lat_a,
    a_HADISST_ATL3, corr_HADISST_ATL3,
    'SSTxALT3 JJ (HADISST)', 'Temperature (°C)'
)

create_plot(
    axes[0,1], lon_a, lat_a,
    a_HADISST_NIÑO, corr_HADISST_NIÑO,
    'SSTxNiño3.4 ND (HADISST)', 'Temperature (°C)'
)

create_plot(
    axes[1,0], lon_b, lat_b,
    a_ERSST_ATL3, corr_ERSST_ATL3,
    'SSTxALT3 JJ (ERSST)', 'Temperature (°C)'
)

create_plot(
    axes[1,1], lon_b, lat_b,
    a_ERSST_NIÑO, corr_ERSST_NIÑO,
    'SSTxNiño3.4 ND (ERSST)', 'Temperature (°C)'
)

plt.tight_layout()
plt.show()


![Correlacion/Slope Anom_SST-index](img/7.png)

In [ ]:
# PCPxATL3 y PCPxNiño3.4
fig, axes = plt.subplots(
    2, 2,
    figsize=(16, 12),
    subplot_kw={'projection': ccrs.PlateCarree(central_longitude=330)}
)

create_plot(
    axes[0,0], lon_pcp, lat_pcp,
    a_pcp_ATL3, corr_pcp_ATL3,
    'PCPxALT3 JJ (NOAA)', 'Precipitation (mm/day)', 'precip'
)

create_plot(
    axes[0,1], lon_pcp, lat_pcp,
    a_pcp_NIÑO, corr_pcp_NIÑO,
    'PCPxNiño3.4 ND (NOAA)', 'Precipitation (mm/day)', 'precip'
)

create_plot(
    axes[1,0], lon_pcp2, lat_pcp2,
    a_pcp2_ATL3, corr_pcp2_ATL3,
    'PCP2xALT3 JJ (NCEP)', 'Precipitation (mm/day)', 'precip'
)

create_plot(
    axes[1,1], lon_pcp2, lat_pcp2,
    a_pcp2_NIÑO, corr_pcp2_NIÑO,
    'PCP2xNiño3.4 ND (NCEP)', 'Precipitation (mm/day)', 'precip'
)
 
plt.tight_layout()
plt.show()


![Correlacion/Slope Anom_PCP-index](img/8.png)

In [ ]:

N = 5

def top5(years, a, b, n=5):
    comb = (a + b) / 2
    idx = np.argsort(comb)[-n:]
    df = pd.DataFrame({"Año": years[idx], "HadISST (°C)": a[idx], "ERSST (°C)": b[idx]})
    return df.sort_values("Año").set_index("Año")

atl3 = top5(years, Temp_ATL3_a, Temp_ATL3_b, N)
niño = top5(years, Temp_Niño_a, Temp_Niño_b, N)

fmt = {"HadISST (°C)": "{:.2f}", "ERSST (°C)": "{:.2f}"}
header_style = [{"selector":"th","props":[("padding-top","2px"),("padding-bottom","2px"),
                                         ("vertical-align","bottom"),("text-align","center")]}]

s_atl3 = atl3.style.format(fmt).set_properties(**{"text-align":"center"}).set_table_styles(header_style)
s_niño = niño.style.format(fmt).set_properties(**{"text-align":"center"}).set_table_styles(header_style)

html = f"""
<div style="text-align:left; font-weight:bold; font-size:15px; margin-bottom:12px;">
Picos de SST media anual (1960–2020)
</div>

<div style="display:flex; justify-content:left; gap:80px;">
  <div>
    <div style="font-weight:bold; margin-bottom:6px;">Atlantic Niño (ATL3):</div>
    {s_atl3.to_html()}
  </div>
  <div>
    <div style="font-weight:bold; margin-bottom:6px;">Pacific Niño (Niño 3.4):</div>
    {s_niño.to_html()}
  </div>
</div>
"""

display(HTML(html))

In [ ]:
TRI_LABELS = ["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"]

# COORDENADAS (Tuplas definidas por el usuario)
ATL3_lats = (-4.0, 4.0)
ATL3_lons = (-20, 0)        # 20W–0E 

NIÑO_lats = (-6.0, 6.0)
NIÑO_lons = (-170, -120)    # 170W–120W 

# Umbrales para clasificación Z
Z_THR_WEAK = 0.5   
Z_THR_MOD  = 1.0
Z_THR_STRONG = 1.5


# 2. FUNCIONES DE EXTRACCIÓN DE DATOS (


def anom_year(year, anoms, year0=1960):
    #Extrae anomalías mensuales (12, lat, lon) para un año específico
    if year < year0:
        raise ValueError(f"No hay datos anteriores a {year0}")
    i = year - year0
    if i < 0 or i >= anoms.shape[0]:
        year_last = year0 + anoms.shape[0] - 1
        raise ValueError(f"Año fuera de rango. Disponible: {year0}–{year_last}")
    return anoms[i, :, :, :]

def mensual_region(mon_field, lat_src, lon_src, lat_min, lat_max, lon_min, lon_max,
                   ddeg=0.25, reducer="area_mean"):

    lon_src_180 = lon_to_180(lon_src)
    lat_t = np.arange(lat_min, lat_max + 1e-9, ddeg)
    lon_t = np.arange(lon_min, lon_max + 1e-9, ddeg)
    out = np.empty(12, dtype=float)

    for m in range(12):
        fld = interp2d_to_target(lat_src, lon_src_180, mon_field[m, :, :], lat_t, lon_t)
        if reducer == "area_mean":
            out[m] = area_weighted_mean(fld, lat_t)
        elif reducer == "mean":
            out[m] = np.nanmean(fld)
        else:
            raise ValueError("reducer must be 'area_mean' or 'mean'")
    return out

def niño_monthly_series(year, lat_min_n, lat_max_n, lon_min_n, lon_max_n): #Sirve para tener las anomalias en la region para cada base de datos


    anom_a_y = anom_year(year, anoms_a, year0=1960)
    anom_b_y = anom_year(year, anoms_b, year0=1960)

    niño_a = mensual_region(anom_a_y, lat_a, lon_a, lat_min_n, lat_max_n, lon_min_n, lon_max_n, ddeg=0.25)
    niño_b = mensual_region(anom_b_y, lat_b, lon_b, lat_min_n, lat_max_n, lon_min_n, lon_max_n, ddeg=0.25)
    return niño_a, niño_b

# =============================================================================
# 3. FUNCIONES MATEMÁTICAS Y ESTADÍSTICAS (STATS HELPERS)
# =============================================================================

def runs(mask, n): #rachas
    m = np.asarray(mask, dtype=bool)
    x = np.r_[False, m, False]
    d = np.diff(x.astype(int))
    starts, ends = np.where(d == 1)[0], np.where(d == -1)[0]
    out = np.zeros_like(m, dtype=bool)
    for s, e in zip(starts, ends):
        if (e - s) >= n:
            out[s:e] = True
    return out

def three_month_running(series_24, year1, year2): #media movil ONI
    s = np.asarray(series_24, dtype=float)
    if s.shape[0] != 24: raise ValueError("Esperaba 24 meses")
    
    tri = np.array([np.nanmean(s[i:i+3]) for i in range(22)])
    centers_idx = np.arange(22) + 1
    centers_month = centers_idx % 12
    
    centers_year = np.where(centers_idx < 12, year1, year2)
    labs = [TRI_LABELS[m] for m in centers_month]
    tri_lab_year = np.array([f"{l}\n{y}" for l, y in zip(labs, centers_year)])
    
    return tri, tri_lab_year, centers_idx

def mark_center_months(valid_tri, centers_idx):
    #marca el centro de cada media movil en el grafico mes a mes como punto valido. es algo visual 
    #antes haciamos las medias sobre ese mes central pero ya no tiene este uso, usamos directamente los meses de picos para eso.
    out = np.zeros(24, dtype=bool)
    for ok, c in zip(valid_tri, centers_idx):
        if ok and 0 <= c < 24: out[c] = True
    return out

def calculate_seasonal_stats(series, sigma_monthly, months_sel):
    """Calcula media, sigma combinada y Z."""
    mean_season = float(np.mean(np.asarray(series)[months_sel]))
    sigs = np.asarray(sigma_monthly, dtype=float)[months_sel]
    sig_season = float(np.sqrt(np.nanmean(sigs**2)))
    
    z = mean_season / sig_season if (sig_season > 0 and np.isfinite(mean_season)) else np.nan
    return mean_season, sig_season, z

def classify_by_z(z, thr_weak=1.0, thr_moderate=1.5, thr_strong=2.0):
    if not np.isfinite(z): return "n/a"
    if z < thr_weak: return "none"
    if z < thr_moderate: return "weak"
    if z < thr_strong: return "moderate"
    return "strong"



In [ ]:
Anoms_ATL3_JJ_a = []

for i in range(61):
    # JJ
    atl3_JJ = anoms_a[i, 5:7, 86:94, 160:180]  # (3, lat, lon)
    Anoms_ATL3_JJ_a.append(np.nanmean(atl3_JJ))

Anoms_ATL3_JJ_a = np.array(Anoms_ATL3_JJ_a)

Anoms_ATL3_JJ_b = []

for i in range(61):
    atl3_JJ = anoms_b[i, 5:7, 42:47, 170:180]
    Anoms_ATL3_JJ_b.append(np.nanmean(atl3_JJ))

Anoms_ATL3_JJ_b = np.array(Anoms_ATL3_JJ_b)


#Niño3.4
Anoms_Niño3_ND_a = []

for i in range(61):
    # ND
    Niño3_ND = anoms_a[i, 10:12, 84:96, 10:60]  # (3, lat, lon)
    Anoms_Niño3_ND_a.append(np.nanmean(Niño3_ND))
    
Anoms_Niño3_ND_a = np.array(Anoms_Niño3_ND_a)

Anoms_Niño3_ND_b = []
def plot_hist_gauss_with_z(ax, data, title, bins=10):
    data = np.asarray(data, dtype=float)
    data = data[np.isfinite(data)]
    n = data.size

    mu = np.mean(data)
    sigma = np.std(data)

    # histograma
    ax.hist(data, bins=bins, density=True, alpha=0.6)

    # gaussiana ajustada
    if sigma > 0:
        x_min, x_max = np.min(data), np.max(data)
        dx = x_max - x_min
        x = np.linspace(x_min - 0.15*dx, x_max + 0.15*dx, 400)
        pdf = (1/(sigma*np.sqrt(2*np.pi))) * np.exp(-0.5*((x-mu)/sigma)**2)
        ax.plot(x, pdf, lw=2)

    # percentiles empíricos
    p84 = np.percentile(data, 84)
    p90 = np.percentile(data, 90)

    # valores Z=±1 en unidades de dato
    z1_val = mu + sigma
    zm1_val = mu - sigma

    # percentil empírico asociado a Z=1 (mu+sigma)
    z1_pct = 100.0 * np.mean(data <= z1_val)

    # líneas guía
    ax.axvline(mu, lw=2, linestyle='-')          # media
    if sigma > 0:
        ax.axvline(zm1_val, lw=2, linestyle='--')  # mu - sigma
        ax.axvline(z1_val,  lw=2, linestyle='--')  # mu + sigma

    ax.axvline(p84, lw=2, linestyle=':')         # P84
    ax.axvline(p90, lw=2, linestyle=':')         # P90

    # anotación compacta (incluye Z y percentil)
    txt = (
        f"N = {n}\n"
        f"μ = {mu:.3f}\n"
        f"σ = {sigma:.3f}\n"
        f"Z=1 → μ+σ = {z1_val:.3f}\n"
        f"Percentil(μ+σ) = {z1_pct:.1f}\n"
        f"P84 = {p84:.3f}\n"
        f"P90 = {p90:.3f}"
    )
    ax.text(0.97, 0.97, txt, transform=ax.transAxes,
            ha="right", va="top")

    ax.set_title(title)
    ax.set_xlabel("Anomalía (°C)")
    ax.set_ylabel("Densidad")
    ax.grid(True, alpha=0.3)

    # leyenda mínima (para entender qué es cada línea)
    # Nota: para no duplicar labels, hacemos handles manuales simples
    from matplotlib.lines import Line2D
    handles = [
        Line2D([0],[0], color='k', lw=2, linestyle='-',  label='μ'),
        Line2D([0],[0], color='k', lw=2, linestyle='--', label='μ±σ (Z=±1)'),
        Line2D([0],[0], color='k', lw=2, linestyle=':',  label='P84 / P90'),
    ]
    ax.legend(handles=handles, loc="upper left", frameon=True)
fig, axs = plt.subplots(1, 2, figsize=(14, 4), sharey=True)

plot_hist_gauss_with_z(
    axs[0],
    Anoms_ATL3_JJ_a,
    title="ATL3 (JJ) – Histograma + Normal + Z y percentiles",
    bins=10
)

plot_hist_gauss_with_z(
    axs[1],
    Anoms_Niño3_ND_a,
    title="Niño (ND) – Histograma + Normal + Z y percentiles",
    bins=10
)

plt.tight_layout()
plt.show()


![Histograma ATL3 y Niño3.4](img/9.png)

In [ ]:
def Detection_Plot_ENSO_ONI_with_intensity(year1, year2):
    criterio = 0.5
    ntri = 5
    months_sel = [10, 11]  # Nov-Dic
    
    # Datos globales
    var_a = np.asarray(variacion_mensual_Niño_a, dtype=float)
    var_b = np.asarray(variacion_mensual_Niño_b, dtype=float)

    # 1. Series temporales (Usando NIÑO_lats / NIÑO_lons)
    niño1_a, niño1_b = niño_monthly_series(year1, NIÑO_lats[0], NIÑO_lats[1], NIÑO_lons[0], NIÑO_lons[1])
    niño2_a, niño2_b = niño_monthly_series(year2, NIÑO_lats[0], NIÑO_lats[1], NIÑO_lons[0], NIÑO_lons[1])
    
    serie_a = np.r_[niño1_a, niño2_a]
    serie_b = np.r_[niño1_b, niño2_b]

    # 2. Detección ONI
    tri_a, tri_lab_year, centers_idx = three_month_running(serie_a, year1, year2)
    tri_b, _,            _           = three_month_running(serie_b, year1, year2)

    valid_tri_a = runs(tri_a >= criterio, ntri)
    valid_tri_b = runs(tri_b >= criterio, ntri)

    valid_month_a = mark_center_months(valid_tri_a, centers_idx)
    valid_month_b = mark_center_months(valid_tri_b, centers_idx)
    valid1_a, valid2_a = valid_month_a[:12], valid_month_a[12:]
    valid1_b, valid2_b = valid_month_b[:12], valid_month_b[12:]

    # 3. Estadísticas
    mean_a, sig_a, z_a = calculate_seasonal_stats(niño1_a, var_a, months_sel)
    mean_b, sig_b, z_b = calculate_seasonal_stats(niño1_b, var_b, months_sel)
    
    cls_a = classify_by_z(z_a, Z_THR_WEAK, Z_THR_MOD, Z_THR_STRONG)
    cls_b = classify_by_z(z_b, Z_THR_WEAK, Z_THR_MOD, Z_THR_STRONG)

    # 4. Plots
    ymin = np.nanmin([np.nanmin(serie_a), np.nanmin(serie_b)])
    ymax = np.nanmax([np.nanmax(serie_a), np.nanmax(serie_b)])
    pad = 0.05 * (ymax - ymin) if (ymax > ymin) else 0.1
    ylim = (ymin - pad, ymax + pad)

    fig = plt.figure(figsize=(14, 10))
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 1])
    ax00 = fig.add_subplot(gs[0, 0])
    ax01 = fig.add_subplot(gs[0, 1])
    ax10 = fig.add_subplot(gs[1, :])

    def plot_year_panel(ax, data_a, data_b, val_a, val_b, title):
        ax.plot(meses, data_a, '-ro', markersize=2, label='HadISST')
        ax.plot(meses, data_b, '-bo', markersize=2, label='ERSST')
        ax.set_title(title)
        ax.set_ylim(ylim)
        ax.grid(True)
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.3f'))
        if np.any(val_a): ax.scatter(np.array(meses)[val_a], np.array(data_a)[val_a], s=18, marker='s', color='red', zorder=3)
        if np.any(val_b): ax.scatter(np.array(meses)[val_b], np.array(data_b)[val_b], s=18, marker='s', color='blue', zorder=3)
        ax.scatter(np.array(meses)[months_sel], np.array(data_a)[months_sel], s=60, marker='D', color='red', zorder=4)
        ax.scatter(np.array(meses)[months_sel], np.array(data_b)[months_sel], s=60, marker='D', color='blue', zorder=4)
        ax.legend()

    plot_year_panel(ax00, niño1_a, niño1_b, valid1_a, valid1_b, f'Niño3.4 anomalies ({year1})')
    plot_year_panel(ax01, niño2_a, niño2_b, valid2_a, valid2_b, f'Niño3.4 anomalies ({year2})')

    # ONI Plot
    x = np.arange(len(tri_a))
    ax10.plot(x, tri_a, '-ro', markersize=3, label='HadISST (3-mo)')
    ax10.plot(x, tri_b, '-bo', markersize=3, label='ERSST (3-mo)')
    ax10.axhline(criterio, linestyle='--', linewidth=1.5)
    ax10.grid(True)
    ax10.set_title('ONI-like detection')
    ax10.set_xticks(x)
    ax10.set_xticklabels(tri_lab_year)
    if np.any(valid_tri_a): ax10.scatter(x[valid_tri_a], tri_a[valid_tri_a], s=35, marker='s', color='red', zorder=3)
    if np.any(valid_tri_b): ax10.scatter(x[valid_tri_b], tri_b[valid_tri_b], s=35, marker='s', color='blue', zorder=3)
    ax10.legend()

    txt_det_a = 'HadISST: Yes' if np.any(valid_tri_a) else 'HadISST: No'
    txt_det_b = 'ERSST: Yes'  if np.any(valid_tri_b) else 'ERSST: No'
    
    fig.text(
        0.5, 0.04,
        f"+{criterio:.1f} °C   5 consecutive overlapping 3-month periods -> {txt_det_a} | {txt_det_b}",
        ha='center', va='bottom', fontsize=10
    )
    fig.text(0.5, 0.01,
        f"HadISST ({year1}-{year2}): mean_season={mean_a:.2f}°C, σ_season={sig_a:.2f}, Z={z_a:.2f}, class={cls_a}   ||   "
        f"ERSST ({year1}-{year2}): mean_season={mean_b:.2f}°C, σ_season={sig_b:.2f}, Z={z_b:.2f}, class={cls_b}",
        ha="center", va="bottom", fontsize=10
    )
    plt.tight_layout(rect=[0, 0.06, 1, 1])
    plt.show()


#POSSIBLE ENSO ENTRE 1972-1973

![Detection_Plot_ENSO_ONI_with_intensity(1972, 1973)](img/10.png)

#POSSIBLE ENSO ENTRE 1982-1983

![Detection_Plot_ENSO_ONI_with_intensity(1982, 1983)](img/11.png)

#POSSIBLE ENSO ENTRE 1987-1988

![Detection_Plot_ENSO_ONI_with_intensity(1987, 1988)](img/12.png)

#Detection_Plot_ENSO_ONI_with_intensity(1986, 1987) 

![Detection_Plot_ENSO_ONI_with_intensity(1987, 1988)](img/13.png)

#POSSIBLE ENSO ENTRE 1997-1998

![Detection_Plot_ENSO_ONI_with_intensity(1997, 1998)](img/14.png)

#POSSIBLE ENSO ENTRE 2015-2016

![Detection_Plot_ENSO_ONI_with_intensity(2015, 2016)](img/15.png)

In [ ]:
def Detection_Plot_ATL3(year0):
    months_sel = [5, 6] # Jun-Jul
    criterio = 0.4
    nmeses = 3
    
    # 1. Obtener Datos (Usando ATL3_lats / ATL3_lons)
    niño_atl_a, niño_atl_b = niño_monthly_series(year0, ATL3_lats[0], ATL3_lats[1], ATL3_lons[0], ATL3_lons[1])
    var_a = np.asarray(variacion_mensual_ATL3_a, dtype=float)
    var_b = np.asarray(variacion_mensual_ATL3_b, dtype=float)

    # 2. Detección
    valid_a = runs(np.asarray(niño_atl_a) >= criterio, nmeses)
    valid_b = runs(np.asarray(niño_atl_b) >= criterio, nmeses)

    # 3. Estadísticas
    mean_a,sig_season_a, z_a = calculate_seasonal_stats(niño_atl_a, var_a, months_sel)
    mean_b, sig_season_b, z_b = calculate_seasonal_stats(niño_atl_b, var_b, months_sel)

    cls_a = classify_by_z(z_a, Z_THR_WEAK, Z_THR_MOD, Z_THR_STRONG)
    cls_b = classify_by_z(z_b, Z_THR_WEAK, Z_THR_MOD, Z_THR_STRONG)


    # 4. Gráficos
    ymin = np.nanmin([np.nanmin(niño_atl_a), np.nanmin(niño_atl_b)])
    ymax = np.nanmax([np.nanmax(niño_atl_a), np.nanmax(niño_atl_b)])
    pad = 0.05 * (ymax - ymin) if (ymax > ymin) else 0.1
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    ax[0].plot(meses, niño_atl_a, '-ro', markersize=2, label='HadISST')
    ax[0].plot(meses, niño_atl_b, '-bo', markersize=2, label='ERSST')
    ax[0].set_ylim(ymin - pad, ymax + pad)
    ax[0].grid()
    ax[0].set_title(f'ATL3 anomalies ({year0})')
    
    if np.any(valid_a): ax[0].scatter(np.array(meses)[valid_a], np.array(niño_atl_a)[valid_a], s=18, marker='s', color='red')
    if np.any(valid_b): ax[0].scatter(np.array(meses)[valid_b], np.array(niño_atl_b)[valid_b], s=18, marker='s', color='blue')
    ax[0].scatter(np.array(meses)[months_sel], np.array(niño_atl_a)[months_sel], s=60, marker='D', color='red')
    ax[0].scatter(np.array(meses)[months_sel], np.array(niño_atl_b)[months_sel], s=60, marker='D', color='blue')
    ax[0].legend()

    ax[1].plot(meses, var_a, '-ro', markersize=2, label='HadISST')
    ax[1].plot(meses, var_b, '-bo', markersize=2, label='ERSST')
    ax[1].set_title('Variability (ATL3)')
    ax[1].grid()
    txt_det_a = 'HadISST: Yes' if np.any(valid_a) else 'HadISST: No'
    txt_det_b = 'ERSST: Yes'  if np.any(valid_b) else 'ERSST: No'
    fig.text(
        0.5, 0.07,
        f"+{criterio:.1f} °C  {nmeses} consecutive months -> {txt_det_a} | {txt_det_b}",
        ha='center', va='bottom', fontsize=10
    )

    fig.text(
        0.5, 0.03,
        f"HadISST({year0})  mean_season={mean_a:.2f}°C, σ_season={sig_season_a:.2f}, Z={z_a:.2f}, class={cls_a}   ||   "
        f"ERSST({year0}) mean_season={mean_b:.2f}°C, σ_season={sig_season_b:.2f}, Z={z_b:.2f}, class={cls_b}",
        ha='center', va='bottom', fontsize=10
    )
    plt.tight_layout(rect=[0, 0.12, 1, 1])
    plt.show()


#POSSIBLE Atlantic Niño en 2010

![Detection_Plot_ATL3(2010)](img/16.png)

#POSSIBLE Atlantic Niño en 2016

![Detection_Plot_ATL3(2016)](img/17.png)

#POSSIBLE Atlantic Niño en 2018

![Detection_Plot_ATL3(2018)](img/18.png)


#POSSIBLE Atlantic Niño en 2019

![Detection_Plot_ATL3(2018)](img/19.png)


#POSSIBLE Atlantic Niño en 2020

![Detection_Plot_ATL3(2029)](img/20.png)

In [ ]:
#NUEVOS TOP 5 PARA ATL3 y Niño3.4, vemos que buscar los años donde en esa zona las anomalias sean mas grandes no sirve de forma anual para ATL3
#Vamos a fijarnos solo en los meses junio julio y agosto de cada año del Altantic Niño y para Septiembre Octubre Noviembre diciembre del Niño pacific.
#ATL3

niño = top5(years, Anoms_Niño3_ND_a,Anoms_Niño3_ND_b, N) #recordamos years=(np.arange(1960, 2021, 1))
atl3 = top5(years, Anoms_ATL3_JJ_a,Anoms_ATL3_JJ_b, N) #recordamos years=(np.arange(1960, 2021, 1))



fmt = {"HadISST (°C)": "{:.2f}", "ERSST (°C)": "{:.2f}"}
header_style = [{"selector":"th","props":[("padding-top","2px"),("padding-bottom","2px"),
                                         ("vertical-align","bottom"),("text-align","center")]}]

s_atl3 = atl3.style.format(fmt).set_properties(**{"text-align":"center"}).set_table_styles(header_style)
s_niño = niño.style.format(fmt).set_properties(**{"text-align":"center"}).set_table_styles(header_style)

html = f"""
<div style="text-align:left; font-weight:bold; font-size:15px; margin-bottom:12px;">
Picos de Anomalias (1960–2020)
</div>

<div style="display:flex; justify-content:left; gap:80px;">
  <div>
    <div style="font-weight:bold; margin-bottom:6px;">Atlantic Niño (ATL3) para Junio/Julio:</div>
    {s_atl3.to_html()}
  </div>
  <div>
    <div style="font-weight:bold; margin-bottom:6px;">Pacific Niño (Niño 3.4) Noviembre/Diciembre:</div>
    {s_niño.to_html()}
  </div>
</div>
"""

display(HTML(html))





#POSSIBLE ENSO ENTRE 1965-1966

![Detection_Plot_ENSO_ONI_with_intensity(1965, 1966)](img/21.png)

#POSSIBLE Atlantic Niño en 1963

![Detection_Plot_ATL3(1963)](img/22.png)

#POSSIBLE Atlantic Niño en 1966

![Detection_Plot_ATL3(1966)](img/23.png)

#POSSIBLE Atlantic Niño en 1968

![Detection_Plot_ATL3(1968)](img/24.png)

#POSSIBLE Atlantic Niño en 1987

![Detection_Plot_ATL3(1987)](img/25.png)

#POSSIBLE Atlantic Niño en 1988

![Detection_Plot_ATL3(1988)](img/26.png)

#POSSIBLE Atlantic Niño en 1996

![Detection_Plot_ATL3(1996)](img/27.png)

In [ ]:
def Plot_global_mean_anoms_with_Z(dataset, year1, year2, year3, year4, months_sel, min_abs_temp=3.0, event=None):
    months_sel = list(months_sel)
    if not months_sel: raise ValueError("Selecciona al menos 1 mes.")
    season_lbl = f"{meses[months_sel[0]]}" if len(months_sel)==1 else f"{meses[months_sel[0]]}–{meses[months_sel[-1]]}"
    
    ds = str(dataset).lower()
    if ds == "a":
        anoms, lons, lats = anoms_a, lon_a, lat_a
        sigma_enso = np.asarray(variacion_mensual_Niño_a, dtype=float)
        sigma_atl3 = np.asarray(variacion_mensual_ATL3_a, dtype=float)
        pick, ds_lbl = 0, "HadISST"
    elif ds == "b":
        anoms, lons, lats = anoms_b, lon_b, lat_b
        sigma_enso = np.asarray(variacion_mensual_Niño_b, dtype=float)
        sigma_atl3 = np.asarray(variacion_mensual_ATL3_b, dtype=float)
        pick, ds_lbl = 1, "ERSST"
    else:
        raise ValueError("dataset debe ser 'a' o 'b'.")

    # Helper Geográfico
    def fix_lon_field(ln, fld):
        ln = np.asarray(ln)
        if np.nanmax(ln) > 180: ln = ((ln + 180) % 360) - 180
        idx = np.argsort(ln)
        return ln[idx], fld[:, idx]
    
    def centers_to_edges(c):
        mid = 0.5 * (c[:-1] + c[1:])
        return np.concatenate([[c[0] - (mid[0]-c[0])], mid, [c[-1] + (c[-1]-mid[-1])]])

    # --- MODO GIF ---
    if event is not None:
        years_anim = [year1] if (year2 == 0 or year2 is None) else [year1, year2]
        frames = []
        for y in years_anim:
            a_y = anom_year(y, anoms, year0=1960)
            for m in range(12): frames.append((y, m, a_y[m, :, :]))

        vmax = float(min_abs_temp)
        fig = plt.figure(figsize=(13, 6.5))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
        ax.add_feature(cfeature.LAND, facecolor="lightgray")
        
        y0, m0, f0 = frames[0]
        lons_fix, f0_fix = fix_lon_field(lons, f0)
        lats_fix = lats[::-1] if (len(lats)>1 and lats[1]<lats[0]) else lats
        if len(lats)>1 and lats[1]<lats[0]: f0_fix = f0_fix[::-1, :]

        mesh = ax.pcolormesh(centers_to_edges(lons_fix), centers_to_edges(lats_fix), np.ma.masked_invalid(f0_fix),
                             transform=ccrs.PlateCarree(), shading="auto", vmin=-vmax, vmax=vmax, cmap=plt.cm.RdBu_r)
        
        plt.colorbar(mesh, ax=ax, orientation="horizontal", pad=0.06).set_label("SST anomaly (°C)")
        title = ax.set_title(f"{ds_lbl} | {event} | {y0} {meses[m0]}")
        
        def update(i):
            y, m, f = frames[i]
            _, f_fix = fix_lon_field(lons, f)
            if len(lats)>1 and lats[1]<lats[0]: f_fix = f_fix[::-1, :]
            mesh.set_array(np.ma.masked_invalid(f_fix).ravel())
            title.set_text(f"Global SST anomalies | {event} | {y} {meses[m]} | {ds_lbl}")
            return [mesh, title]
            
        ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=500)
        out_gif = (f"global_SST_anoms_{ds_lbl}_{event.replace(' ', '_')}_{years_anim[0]}"+ (f"_{years_anim[1]}" if len(years_anim) == 2 else "") + ".gif")
        ani.save(out_gif, writer=animation.PillowWriter(fps=2))
        plt.close(fig)
        display(Image(filename=out_gif))
        return out_gif

    # --- MODO 2x2 ---
    years = [year1, year2, year3, year4]
    fields, Z_enso, Z_atl3 = [], [], []

    for y in years:
        a_y = anom_year(y, anoms, year0=1960)
        fields.append(np.mean(a_y[months_sel, :, :], axis=0))
        
        # Uso de NIÑO_lats/lons y ATL3_lats/lons
        e_a, e_b = niño_monthly_series(y, NIÑO_lats[0], NIÑO_lats[1], NIÑO_lons[0], NIÑO_lons[1])
        _, _, zE = calculate_seasonal_stats((e_a if pick==0 else e_b), sigma_enso, months_sel)
        Z_enso.append(zE)

        a_a, a_b = niño_monthly_series(y, ATL3_lats[0], ATL3_lats[1], ATL3_lons[0], ATL3_lons[1])
        _, _, zA = calculate_seasonal_stats((a_a if pick==0 else a_b), sigma_atl3, months_sel)
        Z_atl3.append(zA)

    vmax = float(min_abs_temp)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12), subplot_kw={"projection": ccrs.PlateCarree()})
    
    for ax, fld, y, zE, zA in zip(axes.ravel(), fields, years, Z_enso, Z_atl3):
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.LAND, facecolor="lightgray")
        fill = ax.contourf(lons, lats, fld, levels=np.linspace(-vmax, vmax, 13),
                           cmap=plt.cm.RdBu_r, transform=ccrs.PlateCarree(), extend="both")
        ax.set_title(f"{y} | Z_ENSO={zE:.2f}  Z_ATL3={zA:.2f}")
        plt.colorbar(fill, ax=ax, orientation="horizontal", pad=0.05).set_label("SST anomaly (°C)")

    fig.suptitle(f"Global SST anomalies ({season_lbl}) — {ds_lbl}", fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()


#FIGURA YEAR-CRITERIA  

![Plot_global_mean_anoms_with_Z("a", 1963, 1968, 1987, 1996, months_sel=[5,6], min_abs_temp=1.5, event=None) ](img/28.png)

#Altanic Niño in 1960-1996-1968-1963

![Plot_global_mean_anoms_with_Z("b", 1960,1996 ,1968 ,1963 , [5,6], min_abs_temp=1.5)](img/29.png)

In [ ]:
#Plot_global_mean_anoms_with_Z("a", 1963, 0, 1972, 2015, months_sel=[10,11], min_abs_temp=2, event="Atlantic Niño")

#Pacific Niño in 1960-1987-1972-2015


![Plot_global_mean_anoms_with_Z("a", 1960, 1987, 1972, 2015, months_sel=[10,11], min_abs_temp=3)](img/30.png)


In [ ]:
#Plot_global_mean_anoms_with_Z("a", 2015, 2016, 1972, 2015, months_sel=[10,11], min_abs_temp=3, event="Pacific Niño")
